# Project 7: E-commerce Product Performance Diagnosis
The objective of this analysis is to process a large transactional dataset from an e-commerce business to identify the top 5 best-performing and bottom 5 worst-performing products by revenue. This will provide actionable recommendations for inventory and marketing teams.

In [174]:
import pandas as pd

# Load the original dataset
df_original = pd.read_csv('online_retail_II.csv')

# Display the first few rows for an initial overview
display(df_original.head())

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1. Data Cleaning and Pre-processing
Before any analysis, we must clean the raw data to ensure its quality and integrity. This process involves handling null values in critical columns, and filtering out invalid transactions such as returns (negative quantity) or items with zero price.

In [175]:
# Chaining the cleaning operations for an efficient workflow
df_clean = (
    df_original
    .dropna(subset=['Description'])          # Remove rows with no product description
    .loc[df_original['Quantity'] > 0]        # Filter for sales only (positive quantity)
    .loc[df_original['Price'] > 0]           # Filter for items with a positive price
    .copy()                                  # Create an explicit copy to avoid warnings
)

# Check the info of the cleaned DataFrame
print("DataFrame structure after cleaning:")
df_clean.info()

DataFrame structure after cleaning:
<class 'pandas.core.frame.DataFrame'>
Index: 1041671 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1041671 non-null  object 
 1   StockCode    1041671 non-null  object 
 2   Description  1041671 non-null  object 
 3   Quantity     1041671 non-null  int64  
 4   InvoiceDate  1041671 non-null  object 
 5   Price        1041671 non-null  float64
 6   Customer ID  805549 non-null   float64
 7   Country      1041671 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 71.5+ MB


## 2. Feature Engineering: Creating the 'Revenue' Column
To analyze performance based on revenue, we first need to create a `Revenue` column. This is achieved by multiplying the `Quantity` of each item by its unit `Price`.

In [176]:
df_clean['Revenue'] = df_clean['Quantity'] * df_clean['Price']

# Display the first few rows to verify the new column
display(df_clean[['Description', 'Quantity', 'Price', 'Revenue']].head())

,Description,Quantity,Price,Revenue
0,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,83.4
1,PINK CHERRY LIGHTS,12,6.75,81.0
2,WHITE CHERRY LIGHTS,12,6.75,81.0
3,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,100.8
4,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,30.0


## 3. Analysis: Product Aggregation and Ranking
Here, we group the entire dataset by product description, sum the total revenue for each, and then sort the results to find the top and bottom performers. This aggregation is the core of our analysis.

In [177]:
# Group by product description, sum the revenue, and round the result
revenue_by_product = (
    df_clean
    .groupby('Description')['Revenue']
    .sum()
    .round(2)
)

# Rank the top 5 best-selling products
top_5_revenue = revenue_by_product.sort_values(ascending=False).head(5)

# Rank the 5 worst-selling products
bottom_5_revenue = revenue_by_product.sort_values(ascending=True).head(5)

## 4. Results & Recommendations
The final analysis reveals the top 5 and bottom 5 products by total revenue, providing clear insights for business strategy.

In [178]:
print("🏆 Top 5 Products by Revenue (Best Sellers)")
display(top_5_revenue)

print("\n⚠️ 5 Products with Lowest Revenue (Underperformers)")
display(bottom_5_revenue)

🏆 Top 5 Products by Revenue (Best Sellers)


Description
REGENCY CAKESTAND 3 TIER              344563.25
Manual                                341104.90
DOTCOM POSTAGE                        322657.48
WHITE HANGING HEART T-LIGHT HOLDER    266923.55
PAPER CRAFT , LITTLE BIRDIE           168469.60
Name: Revenue, dtype: float64


⚠️ 5 Products with Lowest Revenue (Underperformers)


Description
PADS TO MATCH ALL CUSHIONS         0.02
PINK HEART CHRISTMAS DECORATION    0.38
HAPPY BIRTHDAY GINGER CAT CARD     0.38
SET 12 COLOURING PENCILS DOILEY    0.65
CAT W SUNGLASSES BLANK CARD        0.76
Name: Revenue, dtype: float64